In [1]:
#Tutorial from http://www.ccnss.org/wp-content/uploads/2016/07/rlfit.pdf

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [71]:
def pavlovian_value(state,prev_val,reward,alpha=0.1):
    val=prev_val[state]+alpha*(reward-prev_val[state])
    prev_val[state]=val
    return val
def instrumental_value(state,action,prev_q,reward,alpha=0.1):
    q=prev_q[state,action]+alpha*(reward-prev_q[state,action])
    prev_q[state,action]=q
    return prev_q
    

In [123]:
def generate_rewards_according_to_state_and_action(state,action):
    reward_lst=[0,-1,1]
    #0 is nogo, 1 is go
    actions_lst=[0,1]
    #Go to win
    if state==0 and action==1:
        reward=np.random.choice(reward_lst,1,p=[0.2,0,0.8])[0]
    if state==0 and action==0:
        reward=0
    #Nogo to win
    if state==1 and action==0:
        reward=np.random.choice(reward_lst,1,p=[0.2,0,0.8])[0]
    if state==1 and action==1:
        reward=0
    #Nogo to avoid loss
    if state==2 and action==0:
        reward=0
    if state==2 and action==1:
        reward=np.random.choice(reward_lst,1,p=[0.2,0.8,0])[0]
    #Go to avoid loss
    if state==3 and action==1:
        reward=0
    if state==3 and action==0:
        reward=np.random.choice(reward_lst,1,p=[0.2,0.8,0])[0]
    return reward

def action_e(state,pavlovian_value,instrumental_value,epsilon=0.01):
    e_lst=[]
    for a in range(0,2):
        if a==0:
            weight=instrumental_value[state,a]
            pr=np.e**weight
        elif a==1:
            weight=instrumental_value[state,a]+epsilon*pavlovian_value[state]
            pr=np.e**weight
        e_lst.append(pr)
    probability=e_lst/np.sum(e_lst)
    return probability
            
def generate_action(state,pavlovian_value,instrumental_value):
    probability=action_e(state,pavlovian_value,instrumental_value)
    action=np.random.choice([0,1],1,p=probability.flatten())[0]
    return action
    
def generate_state_and_reward(pavlovian_value,instrumental_value):
    states=[0,1,2,3]
    state=np.random.choice(states,1)[0]
    action=generate_action(state,pavlovian_value,instrumental_value)
    reward=generate_rewards_according_to_state_and_action(state,action)
    return state,reward,action
print(generate_rewards_according_to_state_and_action(0,1))

0


In [72]:
val_arr=np.zeros((4,))+0.01
q_arr=np.zeros((4,2))+0.01

print(q_arr)

[[0.01 0.01]
 [0.01 0.01]
 [0.01 0.01]
 [0.01 0.01]]


In [130]:
generate_state_and_reward(val_arr,q_arr)

(2, -1, 1)

In [137]:
def experiment():
    val_arr=np.zeros((4,))+0.01
    q_arr=np.zeros((4,2))+0.01
    reward_lst=[]
    state_lst=[]
    action_lst=[]
    for j in range(0,100):
        state,reward,action=generate_state_and_reward(val_arr,q_arr)
        reward_lst.append(reward)
        action_lst.append(action)
        state_lst.append(state)
        print(state)
        val_arr=pavlovian_value(state,val_arr,reward,alpha=0.1)
        q_arr=instrumental_value(state,action,q_arr,reward,alpha=0.1)
        print(q_arr)
    
experiment()


1
0.009000000000000001


IndexError: invalid index to scalar variable.